# **Rede Neural**

In [3]:
class RedeNeural:
    def __init__(self, estrutura):
        self.estrutura = estrutura
        self.pesos = []
        self.vies = []

---
# **Regressão**
---

### Carregamento do Dataset - California Housing Prices

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

def carregar_dados_regressao():
    df = pd.read_csv('housing.csv')
    
    # TODO arrumar depois - ignorando ocean_proximity
    df = df.drop(columns=['ocean_proximity'])

    # entradas (X) e a saída (y)
    X = df.drop(columns=['median_house_value']).values
    y = df['median_house_value'].values.reshape(-1, 1)
    

    # normaliza os dados
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)

    X_treino, X_teste, y_treino, y_teste = train_test_split(
        X, y, test_size=0.2, random_state=42)

    return X_treino, X_teste, y_treino, y_teste, scaler_y

# configura os dados
X_treino, X_teste, y_treino, y_teste, scaler_y = carregar_dados_regressao()


### Treinamento

### Visualização dos Resultados

---
# **Classificação Binária**
---

### Carregamento do Dataset - PIMA Indians Diabetes Dataset

### Treinamento

### Visualização dos Resultados

---
# **Classificação Multiclasse**
---

### Carregamento do Dataset - Iris

### Treinamento

### Visualização dos Resultados